## Load Results

In [9]:
run_code = "09"

# JUPYTER
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import pickle
import utils.plot_utils as plot
import numpy as np
import matplotlib.pyplot as plt
results_dir = os.getcwd() + "/results/" + run_code
outputpath = results_dir + "/output.pickle"
with open(outputpath, "rb") as f:
    df = pickle.load(f)
with open(results_dir + "/opts.pickle", "rb") as f:
    opts = pickle.load(f)
with open(opts["picklepath"], "rb") as f:
    data = pickle.load(f)

## Plots

#### Dimensionality vs time

In [ ]:
fig, ax = plt.subplots()
ax.plot(df["m"], label="embedding dimension, m")
ax.plot(df["KYD"], label="Kaplan-Yorke Dimension")
fig.legend()

#### Loop through time windows: Cao method summary plot

In [ ]:
from utils.load_utils import normalise_shear
for i, row in df.iterrows():
    data_ = normalise_shear(data[row["win_start"]:row["win_end"]].copy())
    plot.summary_calc_m_tau(data_, opts["dynsys_params"]["tau_to_try"], opts["dynsys_params"]["E1_thresh"], row["E1s"], row["E2s"], row["eps_over_L"], row["m"], row["tau"])